In [1]:
%pwd

'/manitou/pmg/projects/bys2107/cp-fnr'

In [2]:
!nvidia-smi

Fri Apr 12 16:21:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000                On | 00000000:41:00.0 Off |                  Off |
| 30%   34C    P8               24W / 300W|      0MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
## evaluation module
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import zero_one_loss
from sklearn.metrics import jaccard_score

In [5]:
from skmultilearn.dataset import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelD
from skmultilearn.ensemble import RakelO
from skmultilearn.ensemble import MajorityVotingClassifier
from skmultilearn.cluster import FixedLabelSpaceClusterer
from skmultilearn.ensemble import LabelSpacePartitioningClassifier
from sklearn.cluster import KMeans
from skmultilearn.cluster import MatrixLabelSpaceClusterer
from sklearn.multiclass import OneVsRestClassifier

In [6]:
%ls

data/                     main.py*                __pycache__/       train.py*
experiments-ecoli.ipynb*  model-baseline-mlp.pth  README.txt*        utils.py*
fun.py*                   models/                 requirements.txt*


In [6]:
gi_data = pd.read_csv("data/cip_ctx_ctz_gen_multi_data.csv",index_col=0)
gi_pheno = pd.read_csv("data/cip_ctx_ctz_gen_pheno.csv",index_col=0)

In [7]:
import random

SEED = 1000

def constructTrainCalTestSplit(X, Y, train_ratio=0.7, calibration_ratio=0.15, test_ratio=0.15):
    assert train_ratio + calibration_ratio + test_ratio == 1
    # Create a list of indices to shuffle
    indices = list(X.index)
    random.Random(SEED).shuffle(indices)
    
    # Calculate split points
    train_split_point = int(len(indices) * train_ratio)
    calibration_split_point = train_split_point + int(len(indices) * calibration_ratio)
    
    # Split indices into training, calibration, and testing sets
    train_indices = indices[:train_split_point]
    calibration_indices = indices[train_split_point:calibration_split_point]
    test_indices = indices[calibration_split_point:]
    
    # Split the dataframes based on the indices
    X_train = X.loc[train_indices]
    Y_train = Y.loc[train_indices]
    X_calibration = X.loc[calibration_indices]
    Y_calibration = Y.loc[calibration_indices]
    X_test = X.loc[test_indices]
    Y_test = Y.loc[test_indices]
    
    X_train = X_train.values
    Y_train = Y_train.values
    X_calibration = X_calibration.values
    Y_calibration = Y_calibration.values
    X_test = X_test.values
    Y_test = Y_test.values
    
    # Print shapes for verification
    print("X_train shape:", X_train.shape)
    print("Y_train shape:", Y_train.shape)
    print("X_calibration shape:", X_calibration.shape)
    print("Y_calibration shape:", Y_calibration.shape)
    print("X_test shape:", X_test.shape)
    print("Y_test shape:", Y_test.shape)

    return X_train, Y_train, X_calibration, Y_calibration, X_test, Y_test

X_train, Y_train, X_calibration, Y_calibration, X_test, Y_test = constructTrainCalTestSplit(gi_data, gi_pheno)

X_train shape: (566, 60936)
Y_train shape: (566, 4)
X_calibration shape: (121, 60936)
Y_calibration shape: (121, 4)
X_test shape: (122, 60936)
Y_test shape: (122, 4)


In [8]:
def eval_metrics(Y_test, prediction, verbose=True):
    if verbose:
        print("Hamming Loss:", hamming_loss(Y_test, prediction))
        print("Accuracy Score:", accuracy_score(Y_test, prediction))
        print("F1 Score (micro):", f1_score(Y_test, prediction, average='micro'))
        print("F1 Score (macro):", f1_score(Y_test, prediction, average='macro'))
        print("Jaccard Score (average='samples'):", jaccard_score(Y_test, prediction, average='samples'))
        print("Jaccard Score (average='macro'):", jaccard_score(Y_test, prediction, average='macro'))
        print("Jaccard Score (average='micro'):", jaccard_score(Y_test, prediction, average='micro'))
        print("Jaccard Score (average=None):", jaccard_score(Y_test, prediction, average=None))
        print("Precision (macro):", precision_score(Y_test, prediction, average='macro'))
        print("Precision (micro):", precision_score(Y_test, prediction, average='micro'))
        print("Recall (micro):", recall_score(Y_test, prediction, average='micro'))
        print("Recall (macro):", recall_score(Y_test, prediction, average='macro'))
        print("Zero-One Loss (normalized):", zero_one_loss(Y_test, prediction, normalize=True))

    return [
        hamming_loss(Y_test, prediction),
        accuracy_score(Y_test, prediction),
        f1_score(Y_test, prediction, average='micro'),
        f1_score(Y_test, prediction, average='macro'),
        jaccard_score(Y_test, prediction, average='samples'),
        jaccard_score(Y_test, prediction, average='macro'),
        jaccard_score(Y_test, prediction, average='micro'),
        jaccard_score(Y_test, prediction, average=None),
        precision_score(Y_test, prediction, average='macro'),
        precision_score(Y_test, prediction, average='micro'),
        recall_score(Y_test, prediction, average='micro'),
        recall_score(Y_test, prediction, average='macro'),
        zero_one_loss(Y_test, prediction, normalize=True)
    ]

In [11]:
classifier = BinaryRelevance(
    classifier = RandomForestClassifier(),
    require_dense = [False, True]
)

# train
classifier.fit(X_train, Y_train)

# predict
prediction = classifier.predict(X_test)

# print classifiers info
print(classifier.model_count_)
print(classifier.partition_)
print(classifier.classifiers_)

4
[0, 1, 2, 3]
[RandomForestClassifier(), RandomForestClassifier(), RandomForestClassifier(), RandomForestClassifier()]


In [12]:
eval_metrics(Y_test, prediction)

Hamming Loss: 0.18442622950819673
Accuracy Score: 0.5573770491803278
F1 Score (micro): 0.7457627118644068
F1 Score (macro): 0.7126897902247975
Jaccard Score (average='samples'): 0.3360655737704918
Jaccard Score (average='macro'): 0.5727271513630418
Jaccard Score (average='micro'): 0.5945945945945946
Jaccard Score (average=None): [0.8        0.62686567 0.53846154 0.3255814 ]
Precision (macro): 0.7087580231065469
Precision (micro): 0.7374301675977654
Recall (micro): 0.7542857142857143
Recall (macro): 0.7222695046814082
Zero-One Loss (normalized): 0.4426229508196722


/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.18442622950819673,
 0.5573770491803278,
 0.7457627118644068,
 0.7126897902247975,
 0.3360655737704918,
 0.5727271513630418,
 0.5945945945945946,
 array([0.8       , 0.62686567, 0.53846154, 0.3255814 ]),
 0.7087580231065469,
 0.7374301675977654,
 0.7542857142857143,
 0.7222695046814082,
 0.4426229508196722]

In [13]:
classifier = ClassifierChain(
    classifier = RandomForestClassifier(),
    require_dense = [False, True],
    order=[3,2,1,0])

# train
classifier.fit(X_train, Y_train)

# predict
predictions = classifier.predict(X_test)

# print classifiers info
print(classifier.classifiers_)

[RandomForestClassifier(), RandomForestClassifier(), RandomForestClassifier(), RandomForestClassifier()]


In [14]:
eval_metrics(Y_test, predictions)

Hamming Loss: 0.28688524590163933
Accuracy Score: 0.45081967213114754
F1 Score (micro): 0.6089385474860336
F1 Score (macro): 0.6051531319708205
Jaccard Score (average='samples'): 0.23975409836065573
Jaccard Score (average='macro'): 0.4357790646853147
Jaccard Score (average='micro'): 0.43775100401606426
Jaccard Score (average=None): [0.38181818 0.484375   0.49230769 0.38461538]
Precision (macro): 0.6402134646962233
Precision (micro): 0.5956284153005464
Recall (micro): 0.6228571428571429
Recall (macro): 0.6496082966931329
Zero-One Loss (normalized): 0.5491803278688525


/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.28688524590163933,
 0.45081967213114754,
 0.6089385474860336,
 0.6051531319708205,
 0.23975409836065573,
 0.4357790646853147,
 0.43775100401606426,
 array([0.38181818, 0.484375  , 0.49230769, 0.38461538]),
 0.6402134646962233,
 0.5956284153005464,
 0.6228571428571429,
 0.6496082966931329,
 0.5491803278688525]

In [57]:
import random
import numpy as np
from sklearn.base import BaseEstimator, MetaEstimatorMixin, ClassifierMixin
from sklearn.utils import validation
from skmultilearn.problem_transform import ClassifierChain

class EnsembleClassifierChain(
 BaseEstimator, MetaEstimatorMixin, ClassifierMixin):
 def __init__(
     self,
     estimator,
     number_of_chains=24,
     threshold=.5,
     max_features=1.0):
     self.number_of_chains = number_of_chains
     self.estimator = estimator
     self.threshold = threshold
     self.max_features = max_features
     self.estimators_ = []
 def fit(self, X, y):
     validation.check_X_y(X, y, multi_output=True)
     y = validation.check_array(y, accept_sparse=True)
     for i in range(self.number_of_chains):
            # the classifier gets cloned internally in classifer chains, so
            # no need to do that here.
         cc = ClassifierChain(self.estimator)
         no_samples = y.shape[0]
            # create random subset for each chain individually
         idx = random.sample(range(no_samples),
                                int(no_samples * self.max_features))
         cc.fit(X[idx, :], y[idx, :])
         self.estimators_.append(cc)
 def predict(self, X):
     validation.check_is_fitted(self, 'estimators_')
     preds = np.array([cc.predict(X) for cc in self.estimators_])
     preds = np.sum(preds, axis=0)
     W_norm = preds.mean(axis=0)
     out = preds / W_norm
     return (out >= self.threshold).astype(int)
     
     
classifier = EnsembleClassifierChain(RandomForestClassifier())

In [59]:
classifier.fit(X_train, Y_train)

# predict
predictions = classifier.predict(X_test)

AttributeError: 'EnsembleClassifierChain' object has no attribute 'classifiers_'

In [61]:
metrics(predictions, Y_test)

Hamming Loss: 0.19262295081967212
Accuracy Score: 0.5409836065573771
F1 Score (micro): 0.7445652173913043
F1 Score (macro): 0.7168325436107139
Jaccard Score (average='samples'): 0.34357923497267756
Jaccard Score (average='macro'): 0.574300221880867
Jaccard Score (average='micro'): 0.5930735930735931
Jaccard Score (average=None): [0.79032258 0.61428571 0.53703704 0.35555556]
Precision (macro): 0.7539237030080538
Precision (micro): 0.7828571428571428
Recall (micro): 0.7098445595854922
Recall (macro): 0.6860370950888193
Zero-One Loss (normalized): 0.4590163934426229


/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define your model architecture
import torch.nn.init as init

# Define your model architecture
class DeepMultiLabelClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DeepMultiLabelClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)

        # Xavier initialization
        self.init_weights()

    def init_weights(self):
        for layer in [self.fc1, self.fc2, self.fc3, self.fc4]:
            if isinstance(layer, nn.Linear):
                init.xavier_uniform_(layer.weight)
                init.constant_(layer.bias, 0.0)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  # Apply sigmoid activation
        return x

# Define your data loading process
def load_data(X, Y, device, batch_size=32):
    X = torch.tensor(X, dtype=torch.float32).to(device)
    Y = torch.tensor(Y, dtype=torch.float32).to(device)
    joined_dataset = TensorDataset(X, Y)
    return DataLoader(joined_dataset, batch_size=batch_size, shuffle=True)

# Define your training function
def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    lossTrain, lossVal = [], []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_train_loss = round(running_loss/len(train_loader.dataset), 5)
        
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        avg_val_loss = round(val_loss/len(val_loader.dataset), 5)
        
        print(f"Epoch {epoch+1}\nTrain Loss: {avg_train_loss}; Val Loss: {avg_val_loss}")
        lossTrain.append(avg_train_loss)
        lossVal.append(avg_val_loss)
    return lossTrain, lossVal

IOStream.flush timed out


### Example usage:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(device)
    # Parameters
    input_size = len(X_train[0])
    output_size = len(Y_train[0])
    hidden_size = 256
    batch_size = 32
    num_epochs = 100
    learning_rate = 0.000005

    # Instantiate model and load data
    model = DeepMultiLabelClassifier(input_size, hidden_size, output_size).to(device)
    train_loader = load_data(X_train, Y_train, device, batch_size)
    val_loader = load_data(X_calibration, Y_calibration, device, batch_size)
    test_loader = load_data(X_test, Y_test, device, batch_size)
    print(model)
    # Train the model
    hist = train_model(model, train_loader, val_loader=val_loader, num_epochs=num_epochs, learning_rate=learning_rate)

In [12]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(device)
    # Parameters
    input_size = len(X_train[0])
    output_size = len(Y_train[0])
    hidden_size = 256
    batch_size = 32
    num_epochs = 100
    learning_rate = 0.000005

    # Instantiate model and load data
    model = DeepMultiLabelClassifier(input_size, hidden_size, output_size).to(device)
    train_loader = load_data(X_train, Y_train, device, batch_size)
    val_loader = load_data(X_calibration, Y_calibration, device, batch_size)
    test_loader = load_data(X_test, Y_test, device, batch_size)
    print(model)
    # Train the model
    hist = train_model(model, train_loader, val_loader=val_loader, num_epochs=num_epochs, learning_rate=learning_rate)

cuda
DeepMultiLabelClassifier(
  (fc1): Linear(in_features=60936, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=4, bias=True)
)
Epoch 1
Train Loss: 0.02136; Val Loss: 0.01942
Epoch 2
Train Loss: 0.01815; Val Loss: 0.02034
Epoch 3
Train Loss: 0.01694; Val Loss: 0.01905
Epoch 4
Train Loss: 0.01604; Val Loss: 0.02007
Epoch 5
Train Loss: 0.01538; Val Loss: 0.01768
Epoch 6
Train Loss: 0.01506; Val Loss: 0.01746
Epoch 7
Train Loss: 0.0143; Val Loss: 0.01739
Epoch 8
Train Loss: 0.01463; Val Loss: 0.01765
Epoch 9
Train Loss: 0.01427; Val Loss: 0.01662
Epoch 10
Train Loss: 0.01379; Val Loss: 0.01769
Epoch 11
Train Loss: 0.01299; Val Loss: 0.01614
Epoch 12
Train Loss: 0.01236; Val Loss: 0.01599
Epoch 13
Train Loss: 0.01219; Val Loss: 0.01592
Epoch 14
Train Loss: 0.01198; Val Loss: 0.01554
Epoch 15
Train Loss: 0.01178; Val Loss: 0.01817
Epoch 1

In [14]:
pred_raw = model(torch.tensor(X_test, dtype=torch.float32).to(device))
pred = torch.round(pred_raw)
eval_metrics(Y_test, pred.to("cpu").detach().numpy())

Hamming Loss: 0.1885245901639344
Accuracy Score: 0.5983606557377049
F1 Score (micro): 0.7513513513513513
F1 Score (macro): 0.7340785200043647
Jaccard Score (average='samples'): 0.3394808743169399
Jaccard Score (average='macro'): 0.5912243494210707
Jaccard Score (average='micro'): 0.6017316017316018
Jaccard Score (average=None): [0.7704918  0.63076923 0.56363636 0.4       ]
Precision (macro): 0.6960091416819849
Precision (micro): 0.7128205128205128
Recall (micro): 0.7942857142857143
Recall (macro): 0.7787563319957881
Zero-One Loss (normalized): 0.4016393442622951


/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.1885245901639344,
 0.5983606557377049,
 0.7513513513513513,
 0.7340785200043647,
 0.3394808743169399,
 0.5912243494210707,
 0.6017316017316018,
 array([0.7704918 , 0.63076923, 0.56363636, 0.4       ]),
 0.6960091416819849,
 0.7128205128205128,
 0.7942857142857143,
 0.7787563319957881,
 0.4016393442622951]

In [ ]:
pred_raw = model(torch.tensor(X_calibration, dtype=torch.float32).to(device))
pred = torch.round(pred_raw)
eval_metrics(Y_calibration, pred.to("cpu").detach().numpy())

In [23]:
# Example usage:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(device)
    # Parameters
    input_size = len(X_train[0])
    output_size = len(Y_train[0])
    hidden_size = 256
    batch_size = 32
    num_epochs = 100
    learning_rate = 0.000005

    # Instantiate model and load data
    model = DeepMultiLabelClassifier(input_size, hidden_size, output_size).to(device)
    train_loader = load_data(X_train, Y_train, device, batch_size)
    val_loader = load_data(X_calibration, Y_calibration, device, batch_size)
    test_loader = load_data(X_test, Y_test, device, batch_size)
    print(model)
    # Train the model
    hist = train_model(model, train_loader, val_loader=val_loader, num_epochs=num_epochs, learning_rate=learning_rate)

cuda
DeepMultiLabelClassifier(
  (fc1): Linear(in_features=60936, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=4, bias=True)
)
Epoch 1
Train Loss: 0.02101; Val Loss: 0.02009
Epoch 2
Train Loss: 0.01834; Val Loss: 0.01883
Epoch 3
Train Loss: 0.0174; Val Loss: 0.01922
Epoch 4
Train Loss: 0.01749; Val Loss: 0.01823
Epoch 5
Train Loss: 0.01596; Val Loss: 0.01762
Epoch 6
Train Loss: 0.01528; Val Loss: 0.01893
Epoch 7
Train Loss: 0.01492; Val Loss: 0.01751
Epoch 8
Train Loss: 0.014; Val Loss: 0.01756
Epoch 9
Train Loss: 0.0138; Val Loss: 0.01651
Epoch 10
Train Loss: 0.01299; Val Loss: 0.01643
Epoch 11
Train Loss: 0.01353; Val Loss: 0.01653
Epoch 12
Train Loss: 0.01279; Val Loss: 0.01625
Epoch 13
Train Loss: 0.01233; Val Loss: 0.01621
Epoch 14
Train Loss: 0.01204; Val Loss: 0.0162
Epoch 15
Train Loss: 0.01202; Val Loss: 0.01564
Epoch 16
Tr

In [26]:
torch.save(model.state_dict(), 'model-baseline-mlp-100ep.pth')

In [27]:
pred_raw = model(torch.tensor(X_calibration, dtype=torch.float32).to(device))
pred = torch.round(pred_raw)
eval_metrics(Y_calibration, pred.to("cpu").detach().numpy())

Hamming Loss: 0.1921487603305785
Accuracy Score: 0.5702479338842975
F1 Score (micro): 0.7465940054495913
F1 Score (macro): 0.7240520708775036
Jaccard Score (average='samples'): 0.3546831955922865
Jaccard Score (average='macro'): 0.5826664297559819
Jaccard Score (average='micro'): 0.5956521739130435
Jaccard Score (average=None): [0.79365079 0.59701493 0.58       0.36      ]
Precision (macro): 0.7044595490716181
Precision (micro): 0.7248677248677249
Recall (micro): 0.7696629213483146
Recall (macro): 0.7501607587814484
Zero-One Loss (normalized): 0.4297520661157025


/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.1921487603305785,
 0.5702479338842975,
 0.7465940054495913,
 0.7240520708775036,
 0.3546831955922865,
 0.5826664297559819,
 0.5956521739130435,
 array([0.79365079, 0.59701493, 0.58      , 0.36      ]),
 0.7044595490716181,
 0.7248677248677249,
 0.7696629213483146,
 0.7501607587814484,
 0.4297520661157025]

In [28]:
pred_raw = model(torch.tensor(X_test, dtype=torch.float32).to(device))
pred = torch.round(pred_raw)
eval_metrics(Y_test, pred.to("cpu").detach().numpy())

Hamming Loss: 0.1864754098360656
Accuracy Score: 0.5901639344262295
F1 Score (micro): 0.7493112947658402
F1 Score (macro): 0.7303400969587208
Jaccard Score (average='samples'): 0.328551912568306
Jaccard Score (average='macro'): 0.5878193648203291
Jaccard Score (average='micro'): 0.5991189427312775
Jaccard Score (average=None): [0.76190476 0.6557377  0.54901961 0.38461538]
Precision (macro): 0.7083333333333333
Precision (micro): 0.723404255319149
Recall (micro): 0.7771428571428571
Recall (macro): 0.7597105424172572
Zero-One Loss (normalized): 0.4098360655737705


/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.1864754098360656,
 0.5901639344262295,
 0.7493112947658402,
 0.7303400969587208,
 0.328551912568306,
 0.5878193648203291,
 0.5991189427312775,
 array([0.76190476, 0.6557377 , 0.54901961, 0.38461538]),
 0.7083333333333333,
 0.723404255319149,
 0.7771428571428571,
 0.7597105424172572,
 0.4098360655737705]

In [41]:
pred_raw = model(torch.tensor(X_calibration, dtype=torch.float32).to(device))
cal_sgmd = np.round(pred_raw.detach().cpu().numpy(), decimals=5)
cal_labels = Y_calibration
pred_raw = model(torch.tensor(X_test, dtype=torch.float32).to(device))
val_sgmd = np.round(pred_raw.detach().cpu().numpy(), decimals=5)
val_labels = Y_test
print("pred cal", type(cal_sgmd), cal_sgmd.shape)
print("true cal", type(cal_labels), cal_labels.shape)
print("pred val", type(val_sgmd), val_sgmd.shape)
print("true val", type(val_labels), val_labels.shape)

pred cal <class 'numpy.ndarray'> (121, 4)
true cal <class 'numpy.ndarray'> (121, 4)
pred val <class 'numpy.ndarray'> (122, 4)
true val <class 'numpy.ndarray'> (122, 4)


In [42]:
def fnr_custom(preds, labels):
    # FNR for class i: FN / FN + TP
    classes = {}
    for j in range(len(preds[0])): # iterate over classes
        FN = 0
        TP = 0
        for i in range(len(preds)): # iterate over exs
            if labels[i,j] == 1:
                if preds[i,j] != 1: FN += 1
                else: TP += 1
        if FN + TP > 0: # if label never appears, dont worry about it
            classes[j] = {"FN": FN, "TP": TP, "cnt": FN + TP}
    # print(classes)
    total = sum(classes[lbl]["cnt"] for lbl in classes)
    res = 0
    for lbl in classes: 
        res += (classes[lbl]["FN"] / total)
    return res
    
# def false_negative_rate(prediction_set, gt_labels):
#     return 1-((prediction_set * gt_labels).sum(axis=1)/gt_labels.sum(axis=1)).mean()

In [60]:
from scipy.optimize import brentq
alpha = 0.05
n = len(X_calibration)
# Run the conformal risk control procedure
def lamhat_threshold(lam): return fnr_custom(cal_sgmd>=lam, cal_labels) - ((n+1)/n*alpha - 1/(n+1))
lamhat = brentq(lamhat_threshold, 0.001, 0.999)
# prediction_sets = val_sgmd >= lamhat

lamhat

0.029940000735976892

In [61]:
prediction_sets = val_sgmd >= lamhat

In [62]:
# Calculate empirical FNR
print(f"The empirical FNR is: {fnr_custom(prediction_sets, val_labels)} and the threshold value is: {lamhat}")

The empirical FNR is: 0.03428571428571429 and the threshold value is: 0.029940000735976892


In [63]:
eval_metrics(Y_test, prediction_sets.astype(int))

Hamming Loss: 0.42418032786885246
Accuracy Score: 0.2540983606557377
F1 Score (micro): 0.620183486238532
F1 Score (macro): 0.614226854443918
Jaccard Score (average='samples'): 0.3620218579234972
Jaccard Score (average='macro'): 0.4449987512487512
Jaccard Score (average='micro'): 0.449468085106383
Jaccard Score (average=None): [0.5        0.49038462 0.4        0.38961039]
Precision (macro): 0.4523264560498603
Precision (micro): 0.45675675675675675
Recall (micro): 0.9657142857142857
Recall (macro): 0.963628820541279
Zero-One Loss (normalized): 0.7459016393442623


/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0.42418032786885246,
 0.2540983606557377,
 0.620183486238532,
 0.614226854443918,
 0.3620218579234972,
 0.4449987512487512,
 0.449468085106383,
 array([0.5       , 0.49038462, 0.4       , 0.38961039]),
 0.4523264560498603,
 0.45675675675675675,
 0.9657142857142857,
 0.963628820541279,
 0.7459016393442623]

In [55]:
# Calculate empirical FNR
print(f"The empirical FNR without thresholding is: {fnr_custom(np.round(val_sgmd), val_labels)} and the threshold value is: 0.5")

The empirical FNR without thresholding is: 0.22285714285714286 and the threshold value is: 0.5


In [156]:
eval_metrics(Y_test, prediction_sets.to("cpu").detach().numpy().astype(int))

Hamming Loss: 0.1864754098360656
Accuracy Score: 0.5573770491803278
F1 Score (micro): 0.7407407407407407
F1 Score (macro): 0.7115786469963089
Jaccard Score (average='samples'): 0.33265027322404367
Jaccard Score (average='macro'): 0.5682736415167643
Jaccard Score (average='micro'): 0.5882352941176471
Jaccard Score (average=None): [0.79661017 0.57746479 0.54901961 0.35      ]
Precision (macro): 0.7214560862865947
Precision (micro): 0.7386363636363636
Recall (micro): 0.7428571428571429
Recall (macro): 0.7126505627650189
Zero-One Loss (normalized): 0.4426229508196722


/burg/home/bys2107/.conda/envs/nlp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [66]:
cal_sgmd
val_sgmd
cal_labels
val_labels[0].shape

(4,)

In [70]:
lambdas[:3].numpy()

array([0.      , 0.001001, 0.002002], dtype=float32)

In [81]:
i = 0
sigmoids = torch.tensor(cal_sgmd[i], dtype=torch.float32)
gts = torch.tensor(cal_labels[i], dtype=torch.float32)
print(sigmoids)
print(gts)
sigmoids == gts

tensor([0.0280, 0.0024, 0.0025, 0.0017])
tensor([0., 0., 0., 0.])


tensor([False, False, False, False])

In [82]:
N = 1000
n = len(cal_labels)
alpha = 0.1
delta = 0.1
lambdas = torch.linspace(0,1,N) # N can be taken to infinity without penalty
losses = torch.zeros((n,N)) # loss for example i with parameter lambdas[j]
for i in range(n): # In reality we parallelize these loops massively
    sigmoids = torch.tensor(cal_sgmd[i], dtype=torch.float32)
    gts = torch.tensor(cal_labels[i], dtype=torch.float32)
    for j in range(N):
        T = sigmoids > lambdas[j] # This is the prediction set
        # print(T)
        set_size = T.float().sum()
        # print(set_size)
        if set_size != 0:
            losses[i,j] = 1 - (T == gts).float().sum()/set_size
risk = losses.mean(dim=0)
pvals = torch.exp(-2*n*(torch.relu(alpha-risk)**2)) # Or the HB p-value
# Fixed-sequence test starting at lambdas[-1] and ending at lambdas[0]
below_delta = (pvals <= delta).float()
valid = torch.tensor([(below_delta[j:].mean() == 1) for j in range(N)])
lambda_hat = lambdas[valid]

In [85]:
lambda_hat

tensor([0.0961, 0.0971, 0.0981, 0.0991, 0.1001, 0.1011, 0.1021, 0.1031, 0.1041,
        0.1051, 0.1061, 0.1071, 0.1081, 0.1091, 0.1101, 0.1111, 0.1121, 0.1131,
        0.1141, 0.1151, 0.1161, 0.1171, 0.1181, 0.1191, 0.1201, 0.1211, 0.1221,
        0.1231, 0.1241, 0.1251, 0.1261, 0.1271, 0.1281, 0.1291, 0.1301, 0.1311,
        0.1321, 0.1331, 0.1341, 0.1351, 0.1361, 0.1371, 0.1381, 0.1391, 0.1401,
        0.1411, 0.1421, 0.1431, 0.1441, 0.1451, 0.1461, 0.1471, 0.1481, 0.1491,
        0.1502, 0.1512, 0.1522, 0.1532, 0.1542, 0.1552, 0.1562, 0.1572, 0.1582,
        0.1592, 0.1602, 0.1612, 0.1622, 0.1632, 0.1642, 0.1652, 0.1662, 0.1672,
        0.1682, 0.1692, 0.1702, 0.1712, 0.1722, 0.1732, 0.1742, 0.1752, 0.1762,
        0.1772, 0.1782, 0.1792, 0.1802, 0.1812, 0.1822, 0.1832, 0.1842, 0.1852,
        0.1862, 0.1872, 0.1882, 0.1892, 0.1902, 0.1912, 0.1922, 0.1932, 0.1942,
        0.1952, 0.1962, 0.1972, 0.1982, 0.1992, 0.2002, 0.2012, 0.2022, 0.2032,
        0.2042, 0.2052, 0.2062, 0.2072, 